[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/b_fd_approx/fd_approx.ipynb)

In [ ]:
# Necessity libraries
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, exp

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Finite difference approximation

## Test the second-order accuracy
This example tests the finite difference formula that we derive in class using the method of undetermined coefficients, 
$$
D_2 u(\bar{x}) = u'(\bar{x}) + \frac{h^2}{3} u'''(\bar{x}) + O(h^3),
$$
is indeed second-order accurate.

### Computing the error

In [ ]:
# Define a function to numerically differentiate
# Choose one that has an exact solution for the first-derivative
def f(z):
    return exp(z)*sin(z)

# Initial step size, and position to evaluate the derivative at
h = 0.1
x = 1

# The exact derivative for error comparison
dfexact = exp(x)*(cos(x)+sin(x))

# Store the results for later analysis
results = []

# Terminate the while-loop when the step size is sufficiently small
while h > 1e-10:

    # Compute the derivative using the finite-difference stencil
    df = (f(x+2*h)+3*f(x)-4*f(x-h))/(6*h)

    # Print the numerical and exact derivatives,
    # and the magnitude of absolute error
    print(h, df, dfexact, abs(df-dfexact))

    # Store the results for later analysis
    results.append((h, df, dfexact, abs(df-dfexact)))

    # Divide the grid spacing by 2
    h *= 0.5

# Extract step sizes and absolute error values
h_values = [r[0] for r in results]
abserror_values = [r[3] for r in results]

### Plotting the results

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), dpi=300)

# Plot the absolute error vs step size
# in both linear and logarithmic scales
ax[0].plot(h_values, abserror_values, color='tab:blue', label='Abs. error (linear)', marker='o')
ax[1].loglog(h_values, abserror_values, color='tab:orange', label='Abs. error (log)', marker='o')

# Formatting
ax[0].set_xlabel('Step size ($h$)')
ax[0].set_ylabel('Magnitude of abs. error')
ax[0].legend(loc='best')
ax[1].set_xlabel('Step size ($h$)')
ax[1].legend(loc='best')

plt.show()

The graph of the results is difficult to interpret (left), because the values of $h$ span many orders of magnitude. A clearer view is achieved by using logartihmic axes (right).

For $h$ larger than $10^{-5}$, the data appears to follow a quadratic scaling behavior, as expected for a second-order scheme. For $h$ smaller than $10^{-5}$ numerical roundoff errors dominate and the results become less accurate.

**Question**: Why does the absolute error increase once the step size decreases below $10^{-5}$?

### Fitting a power law model

In [ ]:
# Only do fitting when h > 10^-5
h_fit = [h for h in h_values if h > 1e-5]
abserror_fit = abserror_values[:len(h_fit)]

# Perform the power law fit (log-log scale)
# y = a h^b —> log(y) = log(a) + b log(h)
log_h = np.log(h_fit)
log_error = np.log(abserror_fit)
coefficients = np.polyfit(log_h, log_error, 1)  # Linear fit in log-log space

# Print results
b, a = coefficients
print(f"Slope: {b}, Intercept: {a}")

In [ ]:
# Overlay the fitted power law
fig, ax = plt.subplots(1, 2, figsize=(8, 4), dpi=300)

# Plot the absolute error vs step size
# in both linear and logarithmic scales
ax[0].plot(h_values, abserror_values, color='tab:blue', label='Abs. error (linear)', marker='o')
ax[1].loglog(h_values, abserror_values, color='tab:orange', label='Abs. error (log)', marker='o')

# Add the overlay power law
# y_fit = exp(a + b log(h))
ax[1].plot(h_fit, np.exp(a + log_h * b), color='tab:green', label='Power law fit', lw=2)

# Formatting
ax[0].set_xlabel('Step size ($h$)')
ax[0].set_ylabel('Magnitude of abs. error')
ax[0].legend(loc='best')
ax[1].set_xlabel('Step size ($h$)')
ax[1].legend(loc='best')

plt.show()

## Compute the coefficients of a finite-difference formula

Find the coefficients of a first-derivative finite-difference formula at $\bar{x}$ using points $\{ \bar{x}-2h, \bar{x}-h, \bar{x}, \bar{x}+h, \bar{x}+2h \}$.

In [ ]:
# Points to use in finite difference stencil
s = [-2, -1, 0, 1, 2]
n = len(s)

# Assemble linear system using the transpose of the Vandermonde matrix
A = np.fliplr(np.vander(s)).T
d = np.zeros((n))
d[1] = 1

# Solve the linear system and print the coefficients
b = np.linalg.solve(A, d)
for i in range(len(s)):
    print("Coeff. of f(x + %gh): %g" % (s[i], b[i]))